In [45]:
using Pkg 
#Pkg.add("quantum")
#using quantum
#Pkg.add("Distributed")
using Distributed

In [46]:
Threads.nthreads()

6

In [47]:
a = zeros(15)
@time Threads.@threads for i in 1:15
    a[i] = Threads.threadid()
end
a

  0.044406 seconds (33.61 k allocations: 1.672 MiB, 336.71% compilation time)


15-element Vector{Float64}:
 1.0
 1.0
 1.0
 2.0
 2.0
 2.0
 4.0
 4.0
 4.0
 3.0
 3.0
 5.0
 5.0
 6.0
 6.0

In [48]:
b = zeros(15) 
@time for k in 1:15 
    b[k] = k 
end
b

  0.000011 seconds


15-element Vector{Float64}:
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
 10.0
 11.0
 12.0
 13.0
 14.0
 15.0

In [49]:

function sum_single(a)
    s = 0
    for i in a
        s += i
    end
    s
end

sum_single(1:1_000_000)

500000500000

In [50]:

function sum_multi_bad(a)
    s = 0
    Threads.@threads for i in a
        s += i
    end
    s
end

sum_multi_bad(1:1_000_000) # Sufre de "data race"

152777306986

In [51]:
chunks = Iterators.partition(a, length(a) ÷ Threads.nthreads())

Base.Iterators.PartitionIterator{Vector{Float64}}([1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 4.0, 4.0, 4.0, 3.0, 3.0, 5.0, 5.0, 6.0, 6.0], 2)

In [52]:
function sum_multi_good(a)
    chunks = Iterators.partition(a, length(a) ÷ Threads.nthreads())
    tasks = map(chunks) do chunk
        Threads.@spawn sum_single(chunk)
    end
    print(tasks)
    chunk_sums = fetch.(tasks)
    return sum_single(chunk_sums), tasks
end
sum_multi_good(1:1_000_000)[2]

Task[Task (runnable, started) @0x00000003202c01a0, Task (done) @0x00000003202c0e20, Task (done) @0x00000003202c0fb0, Task (done) @0x00000003202c1140, Task (done) @0x00000003202c12d0, Task (done) @0x00000003202c1460, Task (done) @0x00000003202c1f50]

7-element Vector{Task}:
 Task (done) @0x00000003202c01a0
 Task (done) @0x00000003202c0e20
 Task (done) @0x00000003202c0fb0
 Task (done) @0x00000003202c1140
 Task (done) @0x00000003202c12d0
 Task (done) @0x00000003202c1460
 Task (done) @0x00000003202c1f50

In [53]:
cd = sum_multi_good(1:1_000_000)[2]

Task[Task (done) @0x000000031f1f4650, Task (done) @0x000000031f1f47e0, Task (done) @0x000000031f1f4970, Task (done) @0x000000031f1f4b00, Task (done) @0x000000031f1f4e20, Task (done) @0x000000031f1f4fb0, Task (done) @0x000000031f1f5140]

7-element Vector{Task}:
 Task (done) @0x000000031f1f4650
 Task (done) @0x000000031f1f47e0
 Task (done) @0x000000031f1f4970
 Task (done) @0x000000031f1f4b00
 Task (done) @0x000000031f1f4e20
 Task (done) @0x000000031f1f4fb0
 Task (done) @0x000000031f1f5140

In [54]:
fetch(cd)

7-element Vector{Task}:
 Task (done) @0x000000031f1f4650
 Task (done) @0x000000031f1f47e0
 Task (done) @0x000000031f1f4970
 Task (done) @0x000000031f1f4b00
 Task (done) @0x000000031f1f4e20
 Task (done) @0x000000031f1f4fb0
 Task (done) @0x000000031f1f5140

In [55]:
fetch.(cd)

7-element Vector{Int64}:
  13888861111
  41666416667
  69443972223
  97221527779
 124999083335
 152776638891
      3999994

In [56]:
proc = Vector{Tuple{Number, Number}}(undef, 10)
Threads.@threads for i in 1:10
    #push!(proc,(Threads.threadid(), i^2))
    proc[i]=(Threads.threadid(), i^2)
end
proc

10-element Vector{Tuple{Number, Number}}:
 (6, 1)
 (6, 4)
 (4, 9)
 (4, 16)
 (5, 25)
 (5, 36)
 (2, 49)
 (2, 64)
 (3, 81)
 (1, 100)

In [57]:
nprocs()

1

In [58]:
workers()

1-element Vector{Int64}:
 1

In [64]:
rmprocs(workers()) # remove all workers expect the main
addprocs(2)
nprocs()

┌ Warning: rmprocs: process 1 not removed
└ @ Distributed /Users/jorgea22/.julia/juliaup/julia-1.11.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Distributed/src/cluster.jl:1049


3

In [66]:
workers()

2-element Vector{Int64}:
 8
 9

In [67]:
# remove all workers expect the main
#rmprocs(workers()) # remove all workers expect the main
nprocs() # numero de procesos

3

In [68]:
@everywhere using SharedArrays

a = SharedArray{Float64}(10)
@distributed for i = 1:10
    a[i] = myid()
end

Task (runnable, started) @0x000000010de48010

In [69]:
a

10-element SharedVector{Float64}:
 8.0
 8.0
 8.0
 8.0
 8.0
 9.0
 9.0
 9.0
 9.0
 9.0

In [70]:
using LinearAlgebra

In [71]:
M = Matrix{Float64}[rand(10,10) for i = 1:10];

singular_values = pmap(svdvals, M);

In [72]:
singular_values

10-element Vector{Vector{Float64}}:
 [5.101507701990382, 1.4539886170245935, 1.2322762394231965, 1.1792291511981339, 0.8246908417197119, 0.683278487885689, 0.5629980276975178, 0.4120286774096411, 0.16886841579940395, 0.05984331543951153]
 [5.22057797242702, 1.478089682806005, 1.260422372342858, 1.2202327251893261, 0.911176104983399, 0.8067395956530558, 0.4842031403933869, 0.40212688329890994, 0.1548868941111883, 0.04453890176818144]
 [5.572545992398896, 1.5383141084236163, 1.0495899082133242, 0.9547804171113077, 0.7893799337818237, 0.7121311742311685, 0.6088243149664556, 0.3162642702717618, 0.17137107427146442, 0.053742549852720145]
 [5.028879945478178, 1.3383219838733895, 1.2137317201583484, 1.1559645117288146, 0.9333423113262536, 0.6775800175591128, 0.5480325059641697, 0.373399869714335, 0.17860321855669004, 0.11158878584427448]
 [5.294857806159031, 1.3750776944816665, 1.1970885467821157, 1.1095722822330618, 0.8410396242706599, 0.6055888132829828, 0.4791159493682871, 0.30480574296019

In [75]:
@everywhere begin
    #using quantum
    using LinearAlgebra
    using SharedArrays
    function test_parallel(a)
        return a^2
    end
end

In [82]:

@sync @distributed for i in 1:10
    println("Worker $(myid()) is running")
    println(test_parallel(i))
end

      From worker 9:	Worker 9 is running
      From worker 9:	36
      From worker 9:	Worker 9 is running
      From worker 9:	49
      From worker 9:	Worker 9 is running
      From worker 9:	64
      From worker 9:	Worker 9 is running
      From worker 9:	81
      From worker 9:	Worker 9 is running
      From worker 9:	100
      From worker 8:	Worker 8 is running
      From worker 8:	1
      From worker 8:	Worker 8 is running
      From worker 8:	4
      From worker 8:	Worker 8 is running
      From worker 8:	9
      From worker 8:	Worker 8 is running
      From worker 8:	16
      From worker 8:	Worker 8 is running
      From worker 8:	25


Task (done) @0x000000032cb8c330

In [84]:
a = SharedArray{Float64}(10)

# Usar pmap para asignar valores
pmap(i -> (a[i] = myid()), 1:10)

10-element Vector{Int64}:
 8
 9
 9
 8
 9
 8
 9
 8
 9
 8